# Store Slice as pickle

### Imports

In [3]:
from pathlib import Path
import pandas as pd
import re
import numpy as np
import pickle

from python_utils.utils import load_object

### General Args

In [4]:
slice_folder_path = Path("../../../data/4pi_re1000_ha300.360_ch28.p1/SliceYZ_x0015/")

### Load

In [5]:
data_path = slice_folder_path / Path("sliceYZ_x0015_fld.dat")

In [6]:
def parse_file_to_dataframes(file_path):
    # Initialize variables to store parsed data
    data_frames = []
    current_snapshot = []
    headers = []

    # Regular expressions to detect headers
    re_title = re.compile(r'title')
    re_variables = re.compile(r'variables')
    re_zone = re.compile(r'zone')

    try:
        # Read the file line by line
        with open(file_path, "r") as file:
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace

                if not line:  # Skip empty lines
                    continue

                if re_title.match(line):
                    if current_snapshot:  # If there's existing data, store it
                        df = pd.DataFrame(current_snapshot, columns=headers)
                        data_frames.append(df)
                        current_snapshot.clear()  # Clear for the next snapshot
                
                elif re_variables.match(line):
                    headers = re.findall(r'"(.*?)"', line)
                
                elif not re_zone.match(line):  # Ignore 'zone' lines
                    try:
                        values = list(map(float, line.split()))
                        current_snapshot.append(values)  # Append float values to the snapshot
                    except ValueError:
                        print(f"Warning: Could not convert line to float values: {line}")

        # Append the last snapshot if there is one
        if current_snapshot:
            df = pd.DataFrame(current_snapshot, columns=headers)
            data_frames.append(df)

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return data_frames


In [7]:
data_frames = parse_file_to_dataframes(data_path)

In [8]:
def get_snapshot(
        data_frames,
        index,
        ):
    """Retrieve a specific snapshot by index."""
    if 0 <= index < len(data_frames):
        return data_frames[index]
    else:
        print("Invalid snapshot index")
        return None


In [9]:
# Example: Access first snapshot
print("N =", len(data_frames))
df_first_snapshot = get_snapshot(
    data_frames,
    0,
    )
n_x_y, n_v = df_first_snapshot.shape
n_y = n_z = np.sqrt(n_x_y)
print(f"n_y = {n_y}, n_z = {n_z}, n_v = {n_v}")
print(df_first_snapshot.head(n=5))

N = 20000
n_y = 46.0, n_z = 46.0, n_v = 10
          y    z   vx   vy   vz        jx        jy   jz         P         F
0 -1.000000 -1.0  0.0  0.0  0.0 -0.202742  0.000000  0.0 -0.000586  1.594463
1 -0.987221 -1.0  0.0  0.0  0.0 -0.122091  2.072063  0.0 -0.000584  1.580085
2 -0.966764 -1.0  0.0  0.0  0.0 -0.080987  3.681530  0.0 -0.000576  1.517575
3 -0.938938 -1.0  0.0  0.0  0.0 -0.255745  3.601412  0.0 -0.000556  1.412722
4 -0.904241 -1.0  0.0  0.0  0.0 -1.052832  2.337604  0.0 -0.000527  1.307453


### Store

In [10]:
pickle_dir = slice_folder_path / Path("pkl/sliceYZ_x0015_fld.pkl")

In [11]:
def save_object(obj, file_path):
    """
    Save a Python object to a file.
    
    :param obj: The Python object to save
    :param file_path: The path of the file where the object will be stored
    """
    try:
        with open(file_path, 'wb') as file:
            pickle.dump(obj, file)
        print(f"Object successfully saved to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving the object: {e}")

# def load_object(file_path):
#     """
#     Load a Python object from a file.
    
#     :param file_path: The path of the file where the object is stored
#     :return: The loaded Python object
#     """
#     try:
#         with open(file_path, 'rb') as file:
#             obj = pickle.load(file)
#         print(f"Object successfully loaded from {file_path}")
#         return obj
#     except Exception as e:
#         print(f"An error occurred while loading the object: {e}")
#         return None
        

In [12]:
save_object(data_frames, pickle_dir)

Object successfully saved to ..\..\..\data\4pi_re1000_ha300.360_ch28.p1\SliceYZ_x0015\pkl\sliceYZ_x0015_fld.pkl


### Verification

In [13]:
# Load the object back from the specified path
loaded_data = load_object(pickle_dir)

# Verify if it matches the original data
print("N =", len(loaded_data))
df_first_snapshot = get_snapshot(
    loaded_data,
    0,
    )
n_x_y, n_v = df_first_snapshot.shape
n_y = n_z = np.sqrt(n_x_y)
print(f"n_y = {n_y}, n_z = {n_z}, n_v = {n_v}")
print(df_first_snapshot.head(n=5))

Object successfully loaded from ..\..\..\data\4pi_re1000_ha300.360_ch28.p1\SliceYZ_x0015\pkl\sliceYZ_x0015_fld.pkl
N = 20000
n_y = 46.0, n_z = 46.0, n_v = 10
          y    z   vx   vy   vz        jx        jy   jz         P         F
0 -1.000000 -1.0  0.0  0.0  0.0 -0.202742  0.000000  0.0 -0.000586  1.594463
1 -0.987221 -1.0  0.0  0.0  0.0 -0.122091  2.072063  0.0 -0.000584  1.580085
2 -0.966764 -1.0  0.0  0.0  0.0 -0.080987  3.681530  0.0 -0.000576  1.517575
3 -0.938938 -1.0  0.0  0.0  0.0 -0.255745  3.601412  0.0 -0.000556  1.412722
4 -0.904241 -1.0  0.0  0.0  0.0 -1.052832  2.337604  0.0 -0.000527  1.307453
